In [52]:
import pandas as pd

# Cleaning Data 

In [53]:
# Lire le fichier Excel
df = pd.read_excel('datastream.xlsx', sheet_name='Conf Board + Como + ISM + NAHB')

In [54]:
# Conversion de la colonne 'Name' en datetime
df['Name'] = pd.to_datetime(df['Name'])

# Conversion des autres colonnes en float
df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)

# Renommage et mise en index
df.rename(columns={'Name': 'Date'}, inplace=True)
df.set_index('Date', inplace=True)

C:\Users\vadim\AppData\Local\Temp\ipykernel_11808\3957598011.py:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      66.0
1      68.0
2      76.0
3      67.0
4      55.0
       ... 
731    76.0
732    76.0
733    68.0
734    62.0
735    47.0
Name: US ISM MANUFACTURERS SURVEY RESULTS: PRICES PAID - SAME VOLN, Length: 736, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [55]:
# Assurez-vous que l'index de 'data' est de type datetime
df.index = pd.to_datetime(df.index)

# Créer un nouvel index avec le 15 de chaque mois en utilisant la fonction pd.date_range
new_index = pd.date_range(start=df.index.min().replace(day=1), end=df.index.max(), freq='MS') + pd.DateOffset(days=14)

# Réindexer 'data' pour aligner les indices au 15 de chaque mois
df.index = new_index

# Vérifier que les indices ont bien été modifiés
print(df.index)

# Maintenant, data et data_summerschool ont le même index, vous pouvez procéder aux calculs de corrélation


DatetimeIndex(['1963-12-15', '1964-01-15', '1964-02-15', '1964-03-15',
               '1964-04-15', '1964-05-15', '1964-06-15', '1964-07-15',
               '1964-08-15', '1964-09-15',
               ...
               '2024-06-15', '2024-07-15', '2024-08-15', '2024-09-15',
               '2024-10-15', '2024-11-15', '2024-12-15', '2025-01-15',
               '2025-02-15', '2025-03-15'],
              dtype='datetime64[ns]', length=736, freq=None)


In [56]:
# Définir la plage de dates
start_date = "1985-01-01"
end_date =  "2024-08-31" # ou pd.Timestamp.today() mais on aurait des valeurs manquantes

# Filtrer le DataFrame sur la plage de dates
data = df.loc[start_date:end_date]

# Calculer le pourcentage de valeurs manquantes
missing_percent = data.isnull().mean() * 100

# Stocker les colonnes à supprimer et leur pourcentage de valeurs manquantes
removed_columns = {col: missing_percent[col] for col in data.columns if missing_percent[col] > 0}

# Supprimer les colonnes avec des valeurs manquantes
data = data.dropna(axis=1)

# Afficher un tableau avec les colonnes supprimées
removed_df = pd.DataFrame(list(removed_columns.items()), columns=["Colonne", "% Valeurs Manquantes"])
removed_df



,Colonne,% Valeurs Manquantes
0,ICE BofA US High Yield 100 Index - Price Rate of RI,50.210084
1,US ISM MANUFACTURERS SURVEY: SUPPLIER DELIVERY NADJ,68.067227
2,US ISM MANUFACTURERS SURVEY: CUSTOMER INVENTORIES INDEX NADJ,30.252101
3,US ISM MANUFACTURERS SURVEY: INVENTORIES INDEX(DISC.) NADJ,31.932773
4,US ISM MANUFACTURERS SURVEY: SUPPLIER DELIVERY(DISC.) SADJ,31.932773
5,US ISM MANUFACTURERS SURVEY: PRICES PAID INDEX(DISC.) SADJ,54.621849


In [57]:
data.isna().sum().sum()

0

In [58]:
# Vérifier que le nombre de mois entre le premier et le dernier index est égal au nombre de lignes
num_months = (data.index[-1].year - data.index[0].year) * 12 + (data.index[-1].month - data.index[0].month + 1)
is_correct = num_months == len(data)
print(f"Nombre de mois: {num_months}, Nombre de lignes: {len(data)}, Vérification: {is_correct}")


Nombre de mois: 476, Nombre de lignes: 476, Vérification: True


In [59]:
data.shape

(476, 49)

# Correlation

In [60]:
data_summerschool = pd.read_excel('datastream.xlsx', sheet_name='SummerSchool')

data_summerschool.drop(columns=["USA (Acc_Slow)"], inplace=True)

# Supprimer la première ligne du DataFrame data_summerschool avec drop
data_summerschool = data_summerschool.drop(data_summerschool.index[0])

# Conversion de la colonne 'dates' en datetime
data_summerschool['dates'] = pd.to_datetime(data_summerschool['dates'])

# Conversion des autres colonnes en float
data_summerschool.iloc[:, 1:] = data_summerschool.iloc[:, 1:].astype(float)

# Mise en index
data_summerschool.set_index('dates', inplace=True)

data_summerschool.shape

(432, 70)

In [61]:
import difflib

# Faire correspondre les colonnes entre data et data_summerschool
cols_data = set(data.columns)
cols_summerschool = set(data_summerschool.columns)
matched_columns = {}
for col in cols_summerschool:
    match = difflib.get_close_matches(col, cols_data, n=1, cutoff=0.9)
    if match:
        matched_columns[col] = match[0]

#Etendre à la mano les derniers matchings
matched_columns['CRB BLS Spot Index (1967=100) - PRICE INDEX'] = "cmdty BLS Commodity Price Ind 'DEAD' - PRICE INDEX"
matched_columns['Gold Bullion LBM U$/Troy Ounce'] = 'Gold Bullion LBM $/t oz DELAY'
matched_columns["Crude Oil-WTI Spot Cushing U$/BBL - DS MID PRICE"] = 'Crude Oil-WTI Spot Cushing U$/BBL'
matched_columns['US ISM MANUFACTURERS SURVEY: PRICE PAID INDEX SADJ	'] = "US ISM MANUFACTURERS SURVEY RESULTS: PRICES PAID - SAME VOLN"

# Vérifier les correspondances trouvées
print("Correspondances trouvées :", len(matched_columns))

# Calculer la corrélation pour chaque variable appariée en alignant les tailles
correlations = []
for old_col, new_col in matched_columns.items():
    if old_col in data_summerschool.columns and new_col in data.columns:
        min_length = min(len(data_summerschool[old_col]), len(data[new_col]))
        correlation_value = data_summerschool[old_col].iloc[:min_length].corr(data[new_col].iloc[:min_length])
        correlations.append([old_col, new_col, correlation_value])

# Créer un DataFrame avec 3 colonnes
correlation_df = pd.DataFrame(correlations, columns=["Colonne Summerschool", "Colonne Data", "Corrélation"])

# Afficher les résultats
pd.set_option('display.max_colwidth', None)
# Trier le DataFrame par la colonne "Corrélation" de manière décroissante
correlation_df_sorted = correlation_df.sort_values(by="Corrélation", ascending=False)
correlation_df


Correspondances trouvées : 49


,Colonne Summerschool,Colonne Data,Corrélation
0,US CONSUMER CONFIDENCE INDEX SADJ,US CONSUMER CONFIDENCE INDEX SADJ,0.999922
1,US CONSUMER CONFIDENCE IN 6 MONTHS - PLANS TO BUY AUTO SADJ,US CONSUMER CONFIDENCE IN 6 MONTHS - PLANS TO BUY AUTO SADJ,1.000000
2,S&P 500 COMPOSITE - PRICE INDEX,S&P 500 COMPOSITE - PRICE INDEX,0.928458
3,US CONSUMER CONFIDENCE IN 6 MONTHS-BUSINESS CONDITIONS BETTER,US CONSUMER CONFIDENCE IN 6 MONTHS-BUSINESS CONDITIONS BETTER,1.000000
4,US INITIAL CLAIMS FOR UNEMPLOYMENT INSURANCE (BCI 5) VOLA,US INITIAL CLAIMS FOR UNEMPLOYMENT INSURANCE (BCI 5) VOLA,-0.640356
5,US CONSUMER CONFIDENCE CURRENTLY - JOBS PLENTIFUL SADJ,US CONSUMER CONFIDENCE CURRENTLY - JOBS PLENTIFUL SADJ,1.000000
6,US CONSUMER CONFIDENCE IN 6 MONTHS - INCOME DECREASED SADJ,US CONSUMER CONFIDENCE IN 6 MONTHS - INCOME DECREASED SADJ,1.000000
7,US CONSUMER CONFIDENCE INDEX - PRESENT SITUATION SADJ,US CONSUMER CONFIDENCE INDEX - PRESENT SITUATION SADJ,1.000000
8,US CONSUMER CONFIDENCE CURRENTLY - JOBS HARD TO FIND SADJ,US CONSUMER CONFIDENCE CURRENTLY - JOBS HARD TO FIND SADJ,1.000000
9,US CONSUMER CONFIDENCE IN 6 MONTHS-PLANS TO BUY REFRIGERATOR,US CONSUMER CONFIDENCE IN 6 MONTHS-PLANS TO BUY REFRIGERATOR,1.000000


On valide toutes les variables. Meme si S&P 500 n'est pas à 1, cela ne veut pas dire qu'il a été révisé mais plutôt que les entreprises qu'il track changent. Pour Crude Oil, ce n'est pas la même variable mais je pense que crude oil n'est pas révisée non plus ; autant la garder pour le moment quitte à la supprimer plus tard.

In [62]:


'''
#Toutes les variables de data ne sont pas matchées (il en manque 2) et certaines sont matchées plus d'une fois. Il faut donc revoir le matching

# Compter le nombre d'apparitions des noms de colonnes dans "Colonne Data" (new_col)
count_new_cols = correlation_df.groupby('Colonne Data').size().reset_index(name='Nombre d\'apparitions')

# Filtrer les colonnes qui apparaissent plus d'une fois
count_new_cols = count_new_cols[count_new_cols['Nombre d\'apparitions'] > 1]

# Afficher le résultat
count_new_cols


'''


correlation_df['Colonne Data'].nunique()

48

In [63]:

# Colonnes non appariées dans data et data_summerschool
unmatched_data_cols = list(set(data.columns) - set(matched_columns.values()))  # Colonnes de data non appariées
unmatched_summerschool_cols = list(set(data_summerschool.columns) - set(matched_columns.keys()))  # Colonnes de data_summerschool non appariées

# Compléter la liste la plus courte avec des NaN pour équilibrer la longueur
max_length = max(len(unmatched_data_cols), len(unmatched_summerschool_cols))

unmatched_data_cols.extend([None] * (max_length - len(unmatched_data_cols)))
unmatched_summerschool_cols.extend([None] * (max_length - len(unmatched_summerschool_cols)))

# Créer un DataFrame avec les résultats
unmatched_df = pd.DataFrame({
    'Colonnes SummerSchool (non appariées)': unmatched_summerschool_cols,
    'Colonnes Data (non appariées)': unmatched_data_cols
    
})

# Afficher le DataFrame
unmatched_df


,Colonnes SummerSchool (non appariées),Colonnes Data (non appariées)
0,spread102,None
1,US ISM MANUFACTURERS SURVEY: INVENTORIES INDEX NADJ,None
2,spread52,None
3,spread103m,None
4,US ISM MANUFACTURERS SURVEY: SUPPLIER DELIVERY INDEX SADJ,None
5,HY-US,None
6,spread23m,None
7,spread21,None
8,BAA-US,None
9,spread13m,None


Certains consensus survey semblent ne plus être mises à jour. Pour le reste des données il faudra les chercher sur FRED

# CSV output

In [64]:
len(data.columns) == data.columns.nunique()

True

In [66]:
data

,S&P 500 COMPOSITE - PRICE INDEX,cmdty BLS Commodity Price Ind 'DEAD' - PRICE INDEX,Gold Bullion LBM $/t oz DELAY,Crude Oil-WTI Spot Cushing U$/BBL,US INITIAL CLAIMS FOR UNEMPLOYMENT INSURANCE (BCI 5) VOLA,US NAHB MARKET SURVEY: NEW SINGLE FAMILY HOME SALES-CURRENTLY,US CONSUMER CONFIDENCE INDEX SADJ,US CONSUMER CONFIDENCE INDEX - EXPECTATIONS SADJ,US CONSUMER CONFIDENCE INDEX - PRESENT SITUATION SADJ,US CONSUMER CONFIDENCE CURRENTLY - JOBS PLENTIFUL SADJ,...,US CONSUMER CONFIDENCE IN 6 MONTHS-PLANS TO BUY VACUUM CLEANER,US CONSUMER CONFIDENCE IN 6 MONTHS-PLANS TO BUY WASHING MACHINE,US NATIONAL ASSOCIATION OF HOME BUILDERS HOUSING MARKET INDEX,US NAHB MARKET SURVEY: NEW SINGLE FAMILY HOME SALES-IN NEXT 6 MO,US NAHB MARKET SURVEY: TRAFFIC OF PROSPECTIVE NEW HOME BUYERS,US ISM PURCHASING MANAGERS INDEX (MFG SURVEY) SADJ,US ISM MANUFACTURERS SURVEY: NEW ORDERS INDEX SADJ,US ISM MANUFACTURERS SURVEY RESULTS: PRICES PAID - SAME VOLN,US ISM MANUFACTURERS SURVEY: PRODUCTION INDEX SADJ,US ISM MANUFACTURERS SURVEY: EMPLOYMENT INDEX SADJ
1985-01-15,179.63,256.90,306.75,26.35,375.0,50.0,102.0,102.5,101.2,19.6,...,5.2,3.8,50.0,59.0,44.0,50.3,51.8,66.0,54.9,44.4
1985-02-15,181.18,251.90,288.75,28.00,388.8,61.0,103.1,103.9,102.0,19.7,...,5.0,3.6,58.0,64.0,48.0,49.9,51.8,75.0,54.2,45.2
1985-03-15,180.66,256.80,330.00,28.30,382.5,56.0,96.1,95.9,96.5,18.2,...,4.6,3.8,54.0,57.0,48.0,47.8,49.1,74.0,51.2,44.2
1985-04-15,179.83,254.10,321.00,27.60,398.3,51.0,104.4,105.1,103.5,19.1,...,4.2,4.5,49.0,56.0,41.0,48.2,51.3,79.0,51.6,43.2
1985-05-15,189.55,246.80,316.00,27.85,393.5,53.0,99.6,99.6,99.6,19.3,...,3.9,3.0,51.0,58.0,42.0,47.1,49.5,76.0,51.9,40.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-15,5035.69,399.23,2296.49,83.49,210.3,57.0,97.5,68.8,140.6,38.4,...,7.4,5.6,51.0,60.0,34.0,48.8,48.7,60.0,50.7,48.2
2024-05-15,5277.51,399.23,2330.65,77.97,222.3,51.0,101.3,74.9,140.8,37.0,...,8.8,5.6,45.0,51.0,30.0,48.5,46.3,63.0,49.6,50.4
2024-06-15,5460.48,399.23,2326.30,82.83,238.8,48.0,97.8,72.8,135.3,35.5,...,8.5,6.5,43.0,47.0,28.0,48.3,48.9,64.0,48.7,48.4
2024-07-15,5522.30,399.23,2421.91,79.36,238.3,46.0,101.9,78.2,133.6,34.1,...,8.5,5.9,41.0,48.0,27.0,47.0,47.6,61.0,46.6,43.6


In [67]:
data.to_csv("data.csv", index=True)